
# Malaria Gene Annotation — Reproducible Workflow

This notebook demonstrates the end-to-end process and discusses dependencies, version pinning, and FAIR principles.

**Keywords**: malaria, bioinformatics, BLASTX, FASTA, reproducibility, FAIR, workflow

## 1. Environment

This notebook sets up a reproducible, conda-managed environment for the malaria annotation workflow. The specified environment includes Python 3.12 alongside essential packages such as Matplotlib, Biopython, and nbformat, ensuring consistency and compatibility throughout the analysis.

```
name: malaria-workflow
channels:
  - conda-forge
dependencies:
  - python=3.12
  - pip=24.2
  - pip:
      - matplotlib==3.9.2
      - biopython==1.83
      - nbformat==5.10.4
```


In [54]:
# Install condacolab
!pip install -q condacolab
import condacolab
condacolab.install()


✨🍰✨ Everything looks OK!


In [60]:
%pip install biopython==1.83 matplotlib==3.9.2 nbformat==5.10.4
import Bio, matplotlib, nbformat
print(Bio.__version__, matplotlib.__version__, nbformat.__version__)




1.83 3.10.0 5.10.4



## 2. Inputs and expectations

We expect two tab-delimited files:

- `malaria.fna` — multi-FASTA with **tab-delimited** headers, e.g.
```
>1_g	length=1143	scaffold=scaffold00001	strand=-
ATGGATATAA...
```
- `malaria.blastx.tab` — BLASTX table with **gene ID in column 1** and **protein description in column 10**.

If a protein description is `null`, that entry is excluded.
```
#queryName	queryLength	firstQueryPos	lastQueryPos	hitName	hitLength	firstHitPos	lastHitPos	frame	hitDescription	numberOfHits	coverage	identity	evalue	score	hspLength	relIdentity
2701_g	1881	1	1806	Q7RHP3	516	1	516	+1	GAF domain protein	10	1.18	256	2e-110	405	608	0.43
```

If a protein description is null, that entry is excluded.



In [44]:

# Adjust these paths as needed for your environment (e.g., Colab, local)
fna = Path("/content/data/malaria.fna")
blast = Path("/content/data/malaria.blastx.tab")
out_path = Path("/content/data/malaria.annotated.fna")

print("FASTA head:", fna if fna.exists() else "(missing)")
print("BLASTX head:", blast if blast.exists() else "(missing)")
if fna.exists():
    print("\\n".join(fna.read_text().splitlines()[:4]))
if blast.exists():
    print("\\n".join(blast.read_text().splitlines()[:2]))



FASTA head: /content/data/malaria.fna
BLASTX head: /content/data/malaria.blastx.tab
>1_g	length=1143	scaffold=scaffold00001	strand=-\nATGGATATAAATAAGAAATATTTTGCTCAAGATAAATTGGAACATCACTATCACCATTTGTTTCATAATAAATTACCTGCAGATGTGGCTGAGGAATTAGCCGCTACGGCTAAGAAATTAGTAGTACCTGGAAAGGGTATCTTGGCTGCTGATGAAGGAGCTCAGACGATAAAGAAACGATTTGACACTATTAAAATTGAGAACAATGTTGAGAATCGAGCAAACTACAGAGATTTGTTATTTGGTGCTAAAGGTTTGGGAAAATTCATTTCTGGAGTTATTTTATTTGAGGAGACCTTGTTTCAAAAAAATGAAGCAGGTGTGCAACTAGTAAAATTATTACATGAGGAAGGTATAATTCCAGGAATTAAAGTTGATAAGGGATTGGTGGATCTTCCTCGATCTGACCATGAAAAGGCAACGCAGGGTTTAGATGGCTTAGCTGAACGATGCCAGGAATATTACAAAGCTGGTGCTCGATTTGCTAAATGGAGAGCAGTTGTAAATGTTGATATAGAACATGGAAAACCCTCCGAGTTGTCTATTCGAGAAACTGCTTGGACTTTGGCTAGATATGCTTCTATTTGTCAGCATCACAGAATTGTACCCATTGTTGAACCAGAAGTTTTAGCAGATGGAAATCACGGTATTGAAGTCTGTGCTCACGTGACTGAAAGAGTATTACATCATGTTTTTGAGGCATTACATCACCATGGTGTTTTATTAGAAGGTGCTTTATTAAAACCAAATATGGTGACCCCTGGTTATGAGTGTCCAAACAAGGCATCACCACATGATGTAGCTTTTTTCACTGTTAGAACTTTGAAAAGATCTGTGCCACCAGCATTACCCGGTGTTGTCTTTTT


## 3. Build dictionary: gene ID → protein description



In [46]:
# Create dictionary mapping gene IDs to protein descriptions
protein_dict = {}
with blast.open('r') as file:
    for line in file:
        if line.startswith('#'):
            continue
        line = line.rstrip('\n')
        fields = line.split('\t')
        if len(fields) < 10:
            # Skip malformed lines
            continue
        gene_id = fields[0]
        protein_description = fields[9]
        if protein_description != 'null':
            protein_dict[gene_id] = protein_description

print(f"Mapped {len(protein_dict)} gene IDs to protein descriptions")
for gene_id, description in list(protein_dict.items())[:5]:
    print(f"Gene ID: {gene_id}, Description: {description}")

Mapped 4357 gene IDs to protein descriptions
Gene ID: 2701_g, Description: GAF domain protein
Gene ID: 2702_g, Description: Putative uncharacterized protein PY03942 
Gene ID: 2703_g, Description: Putative uncharacterized protein
Gene ID: 2704_g, Description: Bet3 transport protein, putative
Gene ID: 2706_g, Description: Exonuclease i, putative



## 4. Annotate FASTA headers and write output



In [47]:
# Robust multi-line FASTA handling
def _wrap_seq(seq: str, width: int = 60) -> str:
    return "\n".join(seq[i:i+width] for i in range(0, len(seq), width))

def _fasta_iter_multi(fp):
    header = None
    seq_chunks = []
    for raw in fp:
        line = raw.rstrip('\n')
        if not line:
            continue
        if line.startswith('>'):
            if header is not None:
                yield header, ''.join(seq_chunks)
            header = line[1:]  # without '>'
            seq_chunks = []
        else:
            seq_chunks.append(''.join(line.split()).upper())
    if header is not None:
        yield header, ''.join(seq_chunks)

# %% Read FASTA, add protein description, and write output
written = 0
with fna.open('r') as file, out_path.open('w') as output:
    for header, seq in _fasta_iter_multi(file):
        fields = header.split('\t')
        gene_id = fields[0]
        if gene_id in protein_dict:
            output.write('>' + header + '\t' + 'protein=' + protein_dict[gene_id] + '\n')
            output.write(_wrap_seq(seq) + '\n')
            written += 1

written, out_path

(4357, PosixPath('/content/data/malaria.annotated.fna'))


## 5. Visualization

We show the first two annotated records from the output FASTA file to verify content.


In [13]:
# 展示输出 FASTA 的前两个完整记录（header + 全序列）
def iter_fasta_records(path):
    header = None
    seq = []
    with open(path, 'r') as f:
        for line in f:
            line = line.rstrip('\n')
            if line.startswith('>'):
                if header is not None:
                    yield header, ''.join(seq)
                header = line[1:]
                seq = []
            else:
                seq.append(line.strip())
        if header is not None:
            yield header, ''.join(seq)

if not out_path.exists():
    print("Output file not found:", out_path)
else:
    shown = 0
    for hdr, seq in iter_fasta_records(out_path):
        print(">" + hdr)
        for i in range(0, len(seq), 60):
            print(seq[i:i+60])
        print()
        shown += 1
        if shown == 2:
            break
    if shown < 2:
        print(f"(Only {shown} records were written to the output file)")


>1_g	length=1143	scaffold=scaffold00001	strand=-	protein=Fructose-bisphosphate aldolase
ATGGATATAAATAAGAAATATTTTGCTCAAGATAAATTGGAACATCACTATCACCATTTG
TTTCATAATAAATTACCTGCAGATGTGGCTGAGGAATTAGCCGCTACGGCTAAGAAATTA
GTAGTACCTGGAAAGGGTATCTTGGCTGCTGATGAAGGAGCTCAGACGATAAAGAAACGA
TTTGACACTATTAAAATTGAGAACAATGTTGAGAATCGAGCAAACTACAGAGATTTGTTA
TTTGGTGCTAAAGGTTTGGGAAAATTCATTTCTGGAGTTATTTTATTTGAGGAGACCTTG
TTTCAAAAAAATGAAGCAGGTGTGCAACTAGTAAAATTATTACATGAGGAAGGTATAATT
CCAGGAATTAAAGTTGATAAGGGATTGGTGGATCTTCCTCGATCTGACCATGAAAAGGCA
ACGCAGGGTTTAGATGGCTTAGCTGAACGATGCCAGGAATATTACAAAGCTGGTGCTCGA
TTTGCTAAATGGAGAGCAGTTGTAAATGTTGATATAGAACATGGAAAACCCTCCGAGTTG
TCTATTCGAGAAACTGCTTGGACTTTGGCTAGATATGCTTCTATTTGTCAGCATCACAGA
ATTGTACCCATTGTTGAACCAGAAGTTTTAGCAGATGGAAATCACGGTATTGAAGTCTGT
GCTCACGTGACTGAAAGAGTATTACATCATGTTTTTGAGGCATTACATCACCATGGTGTT
TTATTAGAAGGTGCTTTATTAAAACCAAATATGGTGACCCCTGGTTATGAGTGTCCAAAC
AAGGCATCACCACATGATGTAGCTTTTTTCACTGTTAGAACTTTGAAAAGATCTGTGCCA
CCAGCATTACCCGGTGTTGTCTTTTTGTCAGGAGGTCAATCAGAAGAAGAAGCCACTC

## 6. Notes on robustness

The original reference assumed **single-line sequences**. This notebook implements a robust **multi-line FASTA** parser that buffers sequence lines until the next header and normalizes whitespace. The core idea:

```text
for each line:
  if header -> flush previous sequence if had hit; start new record
  else -> append to current sequence buffer
```

Output sequences are wrapped to 60 columns for readability.

## 7. Licence

```
MIT License

Copyright (c) 2025 Xing Fan

Permission is hereby granted, free of charge, to any person obtaining a copy
of this software and associated documentation files (the "Software"), to deal
in the Software without restriction, including without limitation the rights
to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
copies of the Software, and to permit persons to whom the Software is
furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all
copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE
SOFTWARE.
```



## 8. FAIR discussion

- Findable:
All data, scripts, and documentation are uploaded to an open GitHub repository: https://github.com/xfan-01/bern02_workflow
. This makes the workflow easy to find, well organized, and version-controlled.

- Accessible:
The repository is open to everyone without restrictions. A license is included to explain how the data and code can be used, changed, and shared.

- Interoperable:
The workflow uses common data formats such as FASTA and TSV, which can be read by many bioinformatics tools. The scripts are written in Python and run in Jupyter Notebook, and the environment setup is provided, making it easy to use with other pipelines.

- Reusable:
The repository includes data, code, and clear notes about the workflow steps, parameter choices, and environment settings. This allows others to reproduce the results and adapt the workflow for their own research.